In [1]:
import pandas as pd
import plotly.express as px
import pandas as pd
import numpy as np
import glob
import os

In [2]:
# import sys
# sys.path.append('../src/data')
# import makedataset
# df = makedataset.create_weather_dataset('../data/raw/Wetterdaten')
# df = create_weather_dataset("../data/raw/Wetterdaten")

In [3]:
wetter = pd.read_parquet('../data/interim/weather_dataset.parquet')
umsatz = pd.read_parquet('../data/interim/revenue_dataset.parquet')

# Aufgabe 1:

## a. Prüfen Sie, ob ein Zusammenhang zwischen der Durchschnittstemperatur und den Umsätzen der Campingplätze nachweisbar ist. 

In [4]:
# Aggregieren der Wetterdaten
weather_daily_avg = wetter.groupby('MESS_DATUM')['TMK'].mean().reset_index()

# Umbenennen der Spalten für eine einheitliche Datumsspalte
weather_daily_avg.rename(columns={'MESS_DATUM': 'Datum', 'TMK': 'Durchschnittliche Temperatur'}, inplace=True)
umsatz.rename(columns={'Zeit': 'Datum', 'Umsatz': 'Umsatz'}, inplace=True)

# Zusammenführen der Datensätze anhand des Datums
merged_df = pd.merge(umsatz[['Datum', 'Umsatz', '5_Auspraegung_Label']], weather_daily_avg, on='Datum', how='inner')

# Überprüfung der ersten Zeilen des zusammengeführten Datensatzes
merged_df.head()


,Datum,Umsatz,5_Auspraegung_Label,Durchschnittliche Temperatur
0,1994-01-01,255.0,Ausschank von Getränken,1.2
1,1994-01-01,92.3,Beherbergung,1.2
2,1994-01-01,68.4,Campingplätze,1.2
3,1994-01-01,87.9,Caterer und sonstige Verpflegungsdienstleistungen,1.2
4,1994-01-01,97.3,Ferienunterkünfte u. ähnliche Beherbergungsstä...,1.2


In [5]:
fig = px.line(merged_df, x='Datum', y=['Umsatz', 'Durchschnittliche Temperatur'], labels={
    'value': 'Werte',
    'variable': 'Variablen'},
    color= '5_Auspraegung_Label', title='Umsatz und durchschnittliche Temperatur über die Zeit')

fig.update_layout(
    yaxis_title='Umsatz',
    yaxis2=dict(title='Durchschnittliche Temperatur', overlaying='y', side='right')
)

fig.show()


- `Trends und Saisonalität:` Es scheint eine saisonale Schwankung in beiden Datenreihen zu geben, was typisch für Campingplätze sein kann, da sowohl das Wetter als auch die Campingaktivitäten stark saisonabhängig sind.
- `Zusammenhang zwischen Temperatur und Umsatz:` Über das Jahr bewegen sich die Linien ähnlich. Dies könnte darauf hindeuten, dass höhere Temperaturen mit höheren Umsätzen korrespondieren und umgekehrt. 


`Es gibt jedoch Ausnahmen:`
1. 'Ausschank von Getränken' sind kontinuierlich seit 1994. Verhält sich jedoch immernoch saisonell.
2. 'Campingplätze' hat einen unerklärlichen Anstieg im Januar jedes Jahres. Vor 2000 gab es nur kleine Anstiege im Januar. Seit 2000 werden diese Ausreißer immer größer
3. Corona. Einbruch im März/ April 2020

## b. Prüfen Sie zu (a) auch, ob es einen Zusammenhang mit zeitlicher Verzögerung gibt, und ermitteln Sie ggf. mit welchem Lag.

In [6]:
merged_df = merged_df[merged_df['5_Auspraegung_Label'] == 'Campingplätze']

In [7]:
# Berechnung der Kreuzkorrelation zwischen Umsatz und Temperatur
lags = np.arange(-30, 31, 1)  # Untersuchung von Lags von -30 bis +30 Tage
cross_correlations = [merged_df['Umsatz'].corr(merged_df['Durchschnittliche Temperatur'].shift(lag)) for lag in lags]

# Ermittlung des Lags mit der höchsten Korrelation
max_corr = max(cross_correlations, key=abs)
optimal_lag = lags[cross_correlations.index(max_corr)]

# Erstellung des interaktiven Kreuzkorrelationsplots mit Plotly Express
fig = px.line(x=lags, y=cross_correlations, title='Kreuzkorrelation zwischen Umsatz und Durchschnittstemperatur',
              labels={'x': 'Lag (Tage)', 'y': 'Korrelationskoeffizient'})
fig.add_vline(x=optimal_lag, line_dash="dash", line_color="red",
              annotation_text=f"Optimaler Lag: {optimal_lag} Tage", annotation_position="top right")

fig.update_layout(showlegend=False)
fig.show()


optimal_lag, max_corr


(-1, 0.6686296139412291)

Die Korrelation von etwa 0.67 bei einem Lag von -1 zeigt an, dass eine höhere Temperatur gestern tendenziell mit einem höheren Umsatz heute verbunden ist. Dieser Befund kann für geschäftliche Entscheidungen nützlich sein, da er eine Grundlage für die Prognose des Umsatzes auf Basis der Wettervorhersage bietet.

## c. Welcher der Wettermesswerte (= Spalten in den gegebenen Wetterdaten) hängt am stärksten mit den Campingumsätzen zusammen? 

In [8]:
wetter.head()

,MESS_DATUM,QN_3,FX,FM,QN_4,RSK,RSKF,SDK,SHK_TAG,NM,...,TMK,UPM,TXK,TNK,TGK,eor,Geogr.Breite,Geogr.Laenge,Stationshoehe,STATIONS_ID
0,1994-01-01,NaN,NaN,NaN,10.0,3.4,7.0,2.5,0.0,3.7,...,2.3,87.0,6.4,-0.2,0.7,eor,49.1017,7.9920,181.00,377
1,1994-01-02,NaN,NaN,NaN,10.0,3.5,6.0,1.7,1.0,4.7,...,5.5,90.0,8.5,-0.8,-4.5,eor,49.1017,7.9920,181.00,377
2,1994-01-03,NaN,NaN,NaN,10.0,8.4,6.0,0.7,0.0,4.7,...,7.2,94.0,8.9,4.9,1.2,eor,49.1017,7.9920,181.00,377
3,1994-01-04,NaN,NaN,NaN,10.0,5.1,6.0,0.3,0.0,4.0,...,6.4,82.0,8.8,4.7,-0.5,eor,49.1017,7.9920,181.00,377
4,1994-01-05,NaN,NaN,NaN,10.0,21.9,6.0,0.0,0.0,6.0,...,6.1,92.0,7.1,4.6,2.4,eor,49.1017,7.9920,181.00,377


In [9]:
umsatz_camping = umsatz[umsatz['5_Auspraegung_Label'] == 'Campingplätze']
# drop all rows with STATIONS_ID = 5792
wetter_zug = wetter[wetter['STATIONS_ID'] != 5792]

In [10]:
daily_weather_df = wetter.groupby('MESS_DATUM').mean().reset_index()

# Merging the revenue data with the weather data on the date column
merged_df = pd.merge(wetter_zug, umsatz_camping, left_on='MESS_DATUM', right_on='Datum', how='left')


C:\Users\Wladi\AppData\Local\Temp/ipykernel_6288/923914827.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [11]:
# Calculate correlation matrix for revenue and weather measurements
correlation_matrix = merged_df.corr()

# Extract correlation values between 'Umsatz' and weather variables
correlation_with_revenue = correlation_matrix['Umsatz'].drop('Umsatz')  # remove self-correlation
correlation_with_revenue_sorted = correlation_with_revenue.sort_values(ascending=False)

correlation_with_revenue_sorted

C:\Users\Wladi\AppData\Local\Temp/ipykernel_6288/3229237931.py:2: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



TXK                   5.870099e-01
TMK                   5.807473e-01
VPM                   5.038770e-01
TNK                   5.021753e-01
TGK                   4.516293e-01
SDK                   4.187352e-01
QN_3                  2.047462e-01
PM                    4.679561e-02
STATIONS_ID          -4.749252e-15
FX                   -1.601272e-02
RSK                  -1.663561e-02
SHK_TAG              -7.486487e-02
FM                   -8.021826e-02
QN_4                 -9.364887e-02
RSKF                 -1.479914e-01
NM                   -1.888775e-01
4_Auspraegung_Code   -1.975137e-01
UPM                  -4.003851e-01
Statistik_Code                 NaN
Name: Umsatz, dtype: float64

Die Korrelationsanalyse zeigt, dass einige Wettermesswerte stärkere positive Korrelationen mit den Umsätzen der Campingplätze aufweisen:

TXK (Tageshöchsttemperatur) zeigt die stärkste Korrelation (0.61).
TMK (Tagesmitteltemperatur) hat eine ähnlich starke Korrelation (0.59).
TNK (Tagestiefsttemperatur), SDK (tägliche Sonnenscheindauer) und VPM (mittlerer Dampfdruck) zeigen ebenfalls deutliche positive Korrelationen.

Diese Ergebnisse legen nahe, dass höhere Temperaturen positiv mit den Umsätzen der Campingplätze korrelieren, was darauf hindeuten könnte, dass gutes Wetter die Campingaktivitäten fördert.

Die negative Korrelation von UPM (relative mittlere Luftfeuchtigkeit) mit den Umsätzen (-0.4) zeigt, dass höhere Feuchtigkeitswerte tendenziell mit niedrigeren Umsätzen zusammenfallen, was vermutlich auf weniger angenehme Wetterbedingungen für das Camping hindeutet.

## d. Welcher Anteil der (saison- und trendbereinigten) Varianz der Campingumsätze lässt sich aus den verfügbaren Wetterinformationen erklären?

seasonelle Aufteilung prozentual 70% Sommer eher weniger Zugspitze

In [12]:
# Laden der Excel-Datei in einen DataFrame
stationen_df = pd.read_excel("../data/raw/Wetterdaten/KL_Tageswerte_Beschreibung_Stationen.xlsx")

# Einzigartige STATIONS_IDs aus gesamt_df extrahieren
unique_stations_ids = wetter.STATIONS_ID.unique()

# Direktes Filtern von stationen_df für Zeilen, deren 'Stations_id' in unique_stations_ids enthalten ist
gesamt_stationen_df = stationen_df[stationen_df['Stations_id'].isin(unique_stations_ids)].copy()

# Bereinigen der Spaltennamen
gesamt_stationen_df.columns = gesamt_stationen_df.columns.str.strip()

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
ersetzungen = {
    'Stationsname': {
        'W¸rzburg': 'Würzburg',
        'Hamburg-Fuhlsb¸ttel': 'Hamburg-Fuhlsbüttel',
        'Bergzabern, Bad': 'Bad Bergzabern'
    },
    'Bundesland': {
        'Baden-W¸rttemberg': 'Baden-Württemberg'
    }
}

for spalte, ersatz_dict in ersetzungen.items():
    for alt, neu in ersatz_dict.items():
        gesamt_stationen_df[spalte] = gesamt_stationen_df[spalte].str.replace(alt, neu, regex=False)

# rename column Stations_id to STATIONS_ID
gesamt_stationen_df.rename(columns={'Stations_id': 'STATIONS_ID'}, inplace=True)

In [ ]:
gesamt_df = pd.merge(wetter, gesamt_stationen_df[['STATIONS_ID', 'Stationsname', 'Bundesland']], on='STATIONS_ID', how='left')

# Feiertage

In [ ]:
import holidays

# Erstellen eines Feiertagskalenders für Deutschland
de_feiertage = holidays.Germany()

# Funktion, um zu überprüfen, ob ein Datum ein Feiertag in Deutschland ist
# Beachten Sie, dass diese Funktion nicht bundeslandspezifisch ist
def ist_feiertag(datum):
    return datum in de_feiertage

# Hinzufügen der "Feiertag"-Spalte zu gesamt_df
gesamt_df['Feiertag'] = gesamt_df['MESS_DATUM'].apply(lambda x: ist_feiertag(x.date()))

# Für die "Ferien"-Spalte benötigen Sie eine spezifische Logik oder Datenquelle
# Hier setzen wir einfach alle Werte auf False als Platzhalter


In [ ]:
gesamt_df

,STATIONS_ID,MESS_DATUM,QN_3,FX,FM,QN_4,RSK,RSKF,SDK,SHK_TAG,...,TGK,eor,Geogr.Breite,Geogr.Laenge,Stationshoehe,Stationsname_x,Stationsname_y,Bundesland,Feiertag,Ferien
0,880,1887-01-01,NaN,NaN,NaN,1.0,0.0,0.0,NaN,0.0,...,NaN,eor,51.7667,14.4333,70.0,Cottbus,Cottbus,Brandenburg,False,False
1,880,1887-01-02,NaN,NaN,NaN,1.0,0.0,0.0,NaN,0.0,...,NaN,eor,51.7667,14.4333,70.0,Cottbus,Cottbus,Brandenburg,False,False
2,880,1887-01-03,NaN,NaN,NaN,1.0,0.0,0.0,NaN,0.0,...,NaN,eor,51.7667,14.4333,70.0,Cottbus,Cottbus,Brandenburg,False,False
3,880,1887-01-04,NaN,NaN,NaN,1.0,0.0,0.0,NaN,0.0,...,NaN,eor,51.7667,14.4333,70.0,Cottbus,Cottbus,Brandenburg,False,False
4,880,1887-01-05,NaN,NaN,NaN,1.0,0.0,0.0,NaN,0.0,...,NaN,eor,51.7667,14.4333,70.0,Cottbus,Cottbus,Brandenburg,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463643,4763,2023-12-27,NaN,NaN,NaN,3.0,0.0,0.0,NaN,0.0,...,-1.4,eor,51.0500,9.9167,275.0,Sontra,Sontra,Hessen,False,False
463644,4763,2023-12-28,NaN,NaN,NaN,3.0,0.0,0.0,NaN,0.0,...,5.3,eor,51.0500,9.9167,275.0,Sontra,Sontra,Hessen,False,False
463645,4763,2023-12-29,NaN,NaN,NaN,3.0,4.6,4.0,NaN,0.0,...,5.5,eor,51.0500,9.9167,275.0,Sontra,Sontra,Hessen,False,False
463646,4763,2023-12-30,NaN,NaN,NaN,3.0,1.3,4.0,NaN,0.0,...,2.6,eor,51.0500,9.9167,275.0,Sontra,Sontra,Hessen,False,False
